In [15]:
import pymongo
import pandas

1- Créer une connexion vers cette collection

In [2]:
con = pymongo.MongoClient("mongodb://193.51.82.104:2343/")

In [3]:
db = con.test

2- Déterminer le nombre exact de restaurants.

In [4]:
db.restaurants.count()

25359

In [5]:
db.restaurants.find_one()

{'_id': ObjectId('58ac16d1a251358ee4ee87dd'),
 'address': {'building': '1007',
  'coord': [-73.856077, 40.848447],
  'street': 'Morris Park Ave',
  'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
   'grade': 'A',
   'score': 2},
  {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
  {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
  {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}

3- Lister les différents quartiers représentés (borough) ainsi que les différents types de cuisine (cuisine).

In [6]:
db.restaurants.distinct("borough")

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Missing']

In [9]:
db.restaurants.distinct("cuisine")[:5]

['Bakery', 'Hamburgers', 'Irish', 'American', 'Jewish/Kosher']

4- Lister les noms des restaurants, ainsi que leur quartier, ne proposant que des soupes (cuisine égale à Soups).

In [10]:
res = db.restaurants.find({ "cuisine": "Soups" }, { "_id": 0, "name": 1, "borough": 1})
list(res)

[{'borough': 'Manhattan', 'name': 'Wichcraft Express'},
 {'borough': 'Staten Island', 'name': 'Original Soupman Of Staten Island'},
 {'borough': 'Manhattan', 'name': 'The Original Soupman'},
 {'borough': 'Manhattan', 'name': 'Peasant Stock'}]

5- Donner les nombre de restaurants ayant eu au moins une fois le grade (dans grades) Z.

In [12]:
res = db.restaurants.find({ "grades.grade": "Z" })
res.count()

1337

6- Calculer le nombre de restaurants pour chaque quartier

In [23]:
res = db.restaurants.aggregate([
    { "$group": { "_id": "$borough", "nb": { "$sum": 1 }}}
])
pandas.DataFrame(list(res)).rename(columns = {"_id": "Quartier", "nb": "Nb restaurants"})

,Quartier,Nb restaurants
0,Missing,51
1,Staten Island,969
2,Brooklyn,6086
3,Bronx,2338
4,Queens,5656
5,Manhattan,10259


7- Donner les cinq types de cuisine les plus présentes dans New-York (avec le plus de restaurants donc)

In [25]:
res = db.restaurants.aggregate([
    { "$group": { "_id": "$cuisine", "nb": { "$sum": 1 }}},
    { "$sort": { "nb": -1 }},
    { "$limit": 5 }
])
pandas.DataFrame(list(res)).rename(columns = {"_id": "Cuisine", "nb": "Nb restaurants"})

,Cuisine,Nb restaurants
0,American,6183
1,Chinese,2418
2,Café/Coffee/Tea,1214
3,Pizza,1163
4,Italian,1069


8- Pour chaque grade possible, donner le nombre d'évaluation avec ce grade, ainsi que le score moyen, minimum et maximum.

In [40]:
res = db.restaurants.aggregate([
    { "$unwind": "$grades" },
    { "$group": { 
        "_id": "$grades.grade", 
        "nb": { "$sum": 1 },
        "moy": { "$avg": "$grades.score" },
        "min": { "$avg": "$grades.score" },
        "max": { "$avg": "$grades.score" }
    }},
    { "$sort": { "_id": 1 }}
])
pandas.DataFrame(list(res), columns = ["_id", "nb", "moy", "min", "max"]).rename(columns = {
    "_id": "Grade", "nb": "Nb restaurants",
    "moy": "Score moyen", "min": "Score  minimum", "max": "Score maximum"
}).round(2)

,Grade,Nb restaurants,Score moyen,Score minimum,Score maximum
0,A,74656,9.02,9.02,9.02
1,B,12603,20.00,20.00,20.00
2,C,3145,30.70,30.70,30.70
3,Not Yet Graded,525,15.85,15.85,15.85
4,P,1197,5.51,5.51,5.51
5,Z,1337,23.70,23.70,23.70


9- Créer un DataFrame contenant les restaurants en ligne et en colonne le score moyen obtenu pour chaque grade possible. Si un restaurant n'a pas eu d'évaluation d'un grade, nous aurons une valeur absente.

In [70]:
res = db.restaurants.aggregate([
    #{ "$limit": 20 },
    { "$unwind": "$grades" },
    { "$group": { 
        "_id": { "name": "$name", "id": "$restaurant_id", "grade": "$grades.grade" },
        "m": { "$avg": "$grades.score" }
    }},
    { "$sort": { "_id.name": 1 }}
])
df = pandas.DataFrame(list(res))
df["Nom"] = [id["name"] for id in df._id]
df["ID"] = [id["id"] for id in df._id]
df["Grade"] = [id["grade"] for id in df._id]
df.head().round(2)

,_id,m,Nom,ID,Grade
0,"{'name': '#1 Garden Chinese', 'id': '41568346'...",7.5,#1 Garden Chinese,41568346,A
1,"{'name': '#1 Garden Chinese', 'id': '41568346'...",24.0,#1 Garden Chinese,41568346,B
2,"{'name': '#1 Me. Nick'S', 'id': '50014031', 'g...",10.0,#1 Me. Nick'S,50014031,A
3,"{'name': '#1 Sabor Latino Restaurant', 'id': '...",27.0,#1 Sabor Latino Restaurant,41336841,Z
4,"{'name': '#1 Sabor Latino Restaurant', 'id': '...",26.0,#1 Sabor Latino Restaurant,41336841,B


In [69]:
pandas.pivot_table(df, index = ['ID', 'Nom'], columns = 'Grade', values = 'm').round(1)

,Grade,A,B,C,Not Yet Graded,P,Z
ID,Nom,,,,,,
30075445,Morris Park Bake Shop,6.8,14.0,NaN,NaN,NaN,NaN
30112340,Wendy'S,10.7,23.0,NaN,NaN,NaN,NaN
30191841,Dj Reynolds Pub And Restaurant,9.2,NaN,NaN,NaN,NaN,NaN
40356018,Riviera Caterer,9.0,NaN,NaN,NaN,NaN,NaN
40356068,Tov Kosher Kitchen,13.0,25.0,NaN,NaN,NaN,20.0
40356151,Brunos On The Boulevard,10.0,NaN,NaN,NaN,NaN,38.0
40356442,Kosher Island,10.5,NaN,NaN,NaN,NaN,NaN
40356483,Wilken'S Fine Food,10.0,NaN,NaN,NaN,NaN,NaN
40356649,Regina Caterers,9.6,NaN,NaN,NaN,NaN,NaN
